In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("catalogo","catalog_comercial")
dbutils.widgets.text("container","silver")
dbutils.widgets.text("esquema","golden")
dbutils.widgets.text("storage","sajhsvdesa01")

In [0]:
container=dbutils.widgets.get("container")
esquema=dbutils.widgets.get("esquema")
catalogo=dbutils.widgets.get("catalogo")
storage=dbutils.widgets.get("storage")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
df_clientes_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/cliente")
df_ejecutivo_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/ejecutivo")
df_cobertura_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/cobertura")
df_medio_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/medio")
df_tipo_medio_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/tipo_medio")
df_producto_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/producto")
df_tipo_producto_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/tipo_producto")
df_grupo_laboral_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/grupo_laboral")
df_ejecutivo_digital_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/ejecutivo_digital")
df_contrato_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/contrato")
df_forma_factura_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/forma_factura")
df_condicion_pago_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/condicion_pago")
df_pedido_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/pedido")
df_pedido_detalle_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/pedido_detalle")
df_orden_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/orden")
df_orden_detalle_silver=spark.read.format("delta").load(f"abfss://{container}@{storage}.dfs.core.windows.net/tables/orden_detalle")

In [0]:
df_join_ordenes_digitales=df_orden_silver.alias("orden")\
    .join(df_medio_silver.alias("medio"),col("orden.cod_medio")==col("medio.cod_medio"),how="left")\
    .join(df_tipo_medio_silver.alias("tipo_medio"),col("medio.cod_tipo_medio")==col("tipo_medio.cod_tipo_medio"),how="left")\
    .join(df_cobertura_silver.alias("cobertura"),(col("cobertura.cod_cobertura")==col("orden.cod_cobertura")) & (col("cobertura.cod_medio")==col("orden.cod_medio")),how="left")\
    .join(df_producto_silver.alias("producto"),(col("producto.cod_producto")==col("orden.cod_producto")) & (col("producto.cod_medio")==col("orden.cod_medio")), how="left")\
    .join(df_pedido_silver.alias("pedido"), (col("pedido.cod_pedido")==col("orden.cod_pedido")), how="left")\
    .join(df_contrato_silver.alias("contrato"),(col("pedido.cod_contrato")==col("contrato.cod_contrato")),how="left")\
    .join(df_pedido_detalle_silver.alias("peddet"),(col("peddet.cod_pedido")==col("pedido.cod_pedido")) & (col("orden.cod_medio")==col("peddet.cod_medio")) & (col("orden.cod_cobertura")==col("peddet.cod_cobertura")) \
        & (col("peddet.cod_producto")==col("orden.cod_producto")),how="left")\
    .join(df_orden_detalle_silver.alias("orddet"),(col("orddet.cod_orden")==col("orden.cod_orden")) & (col("pedido.cod_pedido")==col("orddet.cod_pedido")) & (col("peddet.cod_pedido_detalle")==col("orddet.cod_pedido_detalle")),how="left")\
    .join(df_forma_factura_silver.alias("forma_factura"),col("forma_factura.cod_forma_factura")==col("pedido.cod_forma_factura"),how="left")\
    .join(df_condicion_pago_silver.alias("condicion_pago"),col("condicion_pago.cod_condicion_pago")==col("pedido.cod_condicion_pago"),how="left")\
    .join(df_clientes_silver.alias("cliente"),col("cliente.cod_cliente")==col("pedido.cod_cliente"),how="left")\
    .join(df_clientes_silver.alias("agencia"),(col("agencia.cod_cliente")==col("pedido.cod_agencia")),how="left")\
    .join(df_ejecutivo_silver.alias("ejecutivo"),(col("ejecutivo.cod_ejecutivo")==col("pedido.cod_ejecutivo")), how="left" )\
    .join(df_ejecutivo_silver.alias("supervisor"),(col("ejecutivo.cod_usuario_ejecutivo_padre")==col("supervisor.cod_ejecutivo")),how="left")\
    .join(df_grupo_laboral_silver.alias("grupo_laboral"),(col("grupo_laboral.cod_grupo_laboral")==col("ejecutivo.cod_grupo_laboral")), how="left")\
    .join(df_ejecutivo_digital_silver.alias("ejedigital"),(col("ejedigital.cod_ejecutivo")==col("ejecutivo.cod_ejecutivo")),how="left")\
    .join(df_ejecutivo_silver.alias("ejed"),(col("ejed.cod_ejecutivo")==col("ejecutivo.cod_ejecutivo")),how="left")\
        .where((col("orden.es_pautable")=="N") & (col("orden.monto_soles")>0) & (col("medio.cod_tipo_medio").isin(["002","014","015"])))\
            .select(col("orden.cod_pedido"),col("pedido.fecha_inicio"),col("pedido.fecha_fin"),col("cliente.razon_social"),col("agencia.razon_social").alias("nombre_agencia"),col("pedido.cod_contrato"),col("pedido.moneda"),\
                col("forma_factura.forma_pago"),col("condicion_pago.condicion_pago"),col("ejecutivo.nombre_ejecutivo"),col("supervisor.nombre_ejecutivo").alias("nombre_supervisor"),col("grupo_laboral.nombre_grupo_laboral"),\
                    col("orden.cod_orden"),col("orden.es_pautable"),col("orden.fecha_inicio"),col("orden.fecha_fin"),col("tipo_medio.nombre_tipo_medio"),col("medio.nombre_medio"),col("cobertura.nombre_cobertura"),\
                        col("producto.nombre_producto"),col("orden.monto_soles"),col("orden.porcentaje_apap"),col("orddet.monto_soles").alias("monto_fila_soles"),col("orddet.monto_dolares").alias("monto_fila_dolares"),\
                            col("ejed.nombre_ejecutivo").alias("Ejecutivo_Digital"))



In [0]:
df_join_ordenes_digitales=df_join_ordenes_digitales.withColumn("nombre_agencia",when(col("nombre_agencia").isNull(),"NO AGENCIA").otherwise(col("nombre_agencia")))
df_join_ordenes_digitales=df_join_ordenes_digitales.withColumn("nombre_cobertura",when(col("nombre_cobertura").isNull(),"NACIONAL").otherwise(col("nombre_cobertura")))
df_join_ordenes_digitales=df_join_ordenes_digitales.withColumn("monto_fila_soles",when(col("monto_fila_soles").isNull(),0).otherwise(col("monto_fila_soles")))
df_join_ordenes_digitales=df_join_ordenes_digitales.withColumn("monto_fila_dolares",when(col("monto_fila_dolares").isNull(),0).otherwise(col("monto_fila_dolares")))

In [0]:
def segmentos_eje(columna):
    segmento=None
    if columna=="PAQUETE CONTENT":
        segmento= "CONTENT"
    elif ((columna=="CONTENIDOS") |(columna=="REDES SOCIALES")):
        segmento= "DISPLAY"
    elif columna=="AUDIO":
        segmento= "AUDIO"
    elif (("BANNER" in columna) | ("BANN" in columna)):
        segmento= "DISPLAY"
    elif columna=="E-MAILING":
        segmento= "E-MAILING" 
    elif columna=="VIDEO":
        segmento= "VIDEO"
    else:
        segmento= "SOCIAL MEDIA"
    return segmento                        


In [0]:
segmento_productos_udf=udf(segmentos_eje,StringType())

In [0]:
df_ordenes_digitales=df_join_ordenes_digitales.withColumn("segmento_eje",segmento_productos_udf("nombre_producto"))


In [0]:
#df_ordenes_digitales.display()

In [0]:
df_ordenes_digitales.write.mode("overwrite").insertInto(f"{catalogo}.{esquema}.ordenes_digitales")